In [1]:
from metafid.deriv import Pricing, OptionStrategy
from metafid.data import TSETMC
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import re 
import finpy_tse as fpy
import psycopg
import jdatetime as jdt
import numpy as np
import re
from CONFIG import DB_PASS as PASS
import inspect
from metafid.ta import RetailInstitutionalPower
from metafid.mfw import DB
from metafid.mfw.deriv import OptionStrategyMFW
import pickle

In [2]:
ostg_mfw = OptionStrategyMFW(dbname="metafid", user="postgres", pass_=PASS, ua_table="sigma", ostg_table="derivs_optionstrategy", pct_daily_cp=0.3, interval=10)
ostg_mfw.do_job()

Press Ctrl+C to exit


In [13]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [15]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df


In [23]:
class OptionStrategy:
    def __init__(self, call:pd.DataFrame, put:pd.DataFrame, pct_daily_cp:float=None) -> None:
            self.call = call
            self.put = put
            self.pct_daily_cp = pct_daily_cp
            self.rep_df = pd.DataFrame(columns=["strategy","position","ua", "ua_final", "t", "bs", "max_pot_profit", "max_pot_loss", "break_even", "current_profit", "pct_cp", "pct_daily_cp", "evaluation"]) 
    def same_strike_and_ex_date_on_call_put(self):
        cols = [i for i in self.call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
        def cols_(x):
            if (x.endswith("_x")) and (x.startswith("ua_")):
                return x.replace("_x","")
            elif (x.endswith("_x")):
                return "call_" + x.replace("_x","")
            elif (x.endswith("_y")):
                return "put_" + x.replace("_y","")
            else:
                return x
        df = self.call.merge(self.put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
        df.columns = list(map(cols_, df.columns))
        return df
    
    def rep_columns(self, cols):
         return [i for i in cols if i in self.rep_df.columns]
    
    def position(self, df):
        def replace_(t:str, char):
            for i in char:
                t = t.replace(i, "")
            return t
        df[["strategy","position"]] = df.stg.map(str).str.split("(", expand=True)
        df["position"] = df.position.apply(lambda x: replace_(x, ["'", ")"]))
        return df


    def long_straddle(self, df):
        """
        A long straddle strategy is an options strategy that involves buying a call and a put on the same
        underlying asset with the same strike price and expiration date. The strike price is usually at-the-money
        or close to it. The goal of this strategy is to profit from a very strong move in either direction by the
        underlying asset, often triggered by a newsworthy event.

        :param df: with (strike_price, ua_final, call_buy_price, put_buy_price) columns
        :return:maximum loss: net premium received
                maximum profit: unlimited
                lower break-even: strike price – net premium
                upper break-even: strike price  + net premium
        """
        stg = namedtuple("LongStraddle", "buy buy_at buy_ buy_at_")
        df["stg"] = df.apply(lambda x: stg(buy=x["call_option"], buy_at=x["call_sell_price"], buy_=x["put_option"], buy_at_=x["put_sell_price"]),
                             axis=1)
        df = self.position(df)
        df = df.assign(max_pot_loss=-df.call_sell_price - df.put_sell_price)
        df = df.assign(
            lower_break_even=df.strike_price + df.max_pot_loss,
            upper_break_even=df.strike_price - df.max_pot_loss,
        )
        df["current_profit"] = df.apply(
            lambda x: max(
                x["max_pot_loss"],
                abs(x["ua_final"] - x["strike_price"]) + x["max_pot_loss"],
            ),
            axis=1,
        )
        df = df.assign(pct_cp=df.current_profit / (df.call_sell_price + df.put_sell_price) * 100).round(2)
        df = df.assign(pct_daily_cp=df.pct_cp / df.t).round(2)
        df = df[self.rep_columns(df.columns)]

        if self.pct_daily_cp:
            return df[df.pct_daily_cp > self.pct_daily_cp]
        else:
            return df

    

In [3]:
call = ostg_mfw.data().call
put = ostg_mfw.data().put

In [30]:
test = OptionStrategy(call=call, put=put, pct_daily_cp=-1)
mixed_df = test.same_strike_and_ex_date_on_call_put()
mixed_df = mixed_df[(mixed_df.call_sell_price >0) & (mixed_df.put_sell_price >0)]
mixed_df

,call_option,call_name,call_final,call_value,call_ob_depth,call_sell_vol,call_sell_price,call_buy_price,call_buy_vol,dt,...,put_option,put_name,put_final,put_value,put_ob_depth,put_sell_vol,put_sell_price,put_buy_price,put_buy_vol,put_bs
0,ضخود2033,اختیارخ خودرو-3250-1402/02/13,629,1275744000,3,10,660,136,368,1402-01-20 02:02:52,...,طخود2033,اختیارف خودرو-3250-1402/02/13,1,0,1,200,100,0,0,0
2,ضخود2033,اختیارخ خودرو-3250-1402/02/13,629,1275744000,2,10,657,597,62,1402-01-20 02:02:52,...,طخود2033,اختیارف خودرو-3250-1402/02/13,1,0,1,200,100,0,0,0
3,ضخود2033,اختیارخ خودرو-3250-1402/02/13,629,1275744000,1,50,655,606,19,1402-01-20 02:02:52,...,طخود2033,اختیارف خودرو-3250-1402/02/13,1,0,1,200,100,0,0,0
39,ضخود3061,اختیارخ خودرو-1600-1402/03/03,2100,199500000,1,100,2400,2100,10,1402-01-20 02:02:52,...,طخود3061,اختیارف خودرو-1600-1402/03/03,26,0,1,1,28,0,0,0
63,ضخود2034,اختیارخ خودرو-3500-1402/02/13,466,3519600000,5,10,515,177,1,1402-01-20 02:02:52,...,طخود2034,اختیارف خودرو-3500-1402/02/13,200,0,1,100,250,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,ضهای3006,اختیارخ های وب-2998-1402/03/03,545,6965760152,1,100,519,477,200,1402-01-20 02:02:52,...,طهای3006,اختیارف های وب-2998-1402/03/03,85,749301992,3,238,136,81,101,18
1114,ضهای3006,اختیارخ های وب-2998-1402/03/03,545,6965760152,1,100,519,477,200,1402-01-20 02:02:52,...,طهای3006,اختیارف های وب-2998-1402/03/03,85,749301992,1,210,100,85,1,18
1181,ضسمگا703,اختیارخ سمگا-8500-14020709,2000,0,1,130,2000,556,100,1402-01-20 02:02:52,...,طسمگا703,اختیارف سمگا-8500-14020709,480,0,1,40,980,10,1,536
1186,ضسمگا703,اختیارخ سمگا-8500-14020709,2000,0,2,35,3500,555,1,1402-01-20 02:02:52,...,طسمگا703,اختیارف سمگا-8500-14020709,480,0,1,40,980,10,1,536


In [29]:
test.long_straddle(mixed_df)

,ua,ua_final,t,strategy,position,max_pot_loss,current_profit,pct_cp,pct_daily_cp
39,خودرو,3718,45,LongStraddle,"buy=ضخود3061, buy_at=2400, buy_=طخود3061, buy_...",-2428,-310,-12.77,-0.28
106,فملی,8270,49,LongStraddle,"buy=ضملی3025, buy_at=2700, buy_=طملی3025, buy_...",-2910,-1307,-44.91,-0.92
280,شستا,1410,143,LongStraddle,"buy=ضستا6010, buy_at=290, buy_=طستا6010, buy_a...",-487,-432,-88.71,-0.62
281,شستا,1410,143,LongStraddle,"buy=ضستا6010, buy_at=290, buy_=طستا6010, buy_a...",-495,-440,-88.89,-0.62
282,شستا,1410,143,LongStraddle,"buy=ضستا6010, buy_at=290, buy_=طستا6010, buy_a...",-492,-437,-88.82,-0.62
...,...,...,...,...,...,...,...,...,...
1059,های وب,3225,45,LongStraddle,"buy=ضهای3000, buy_at=1498, buy_=طهای3000, buy_...",-1553,-173,-11.14,-0.25
1064,های وب,3225,45,LongStraddle,"buy=ضهای3000, buy_at=1500, buy_=طهای3000, buy_...",-1555,-175,-11.25,-0.25
1181,سمگا,8540,175,LongStraddle,"buy=ضسمگا703, buy_at=2000, buy_=طسمگا703, buy_...",-2980,-2940,-98.66,-0.56
1186,سمگا,8540,175,LongStraddle,"buy=ضسمگا703, buy_at=3500, buy_=طسمگا703, buy_...",-4480,-4440,-99.11,-0.57


In [14]:
ri = RetailInstitutionalPower(ticker="برکت", start_date="1401-01-01", end_date="1402-01-10")
df_ri = ri.ri_power()

In [ ]:
df_ri["ticker"] = "برکت"
df_ri = df_ri.replace(np.nan, 0)
df_ri[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]] = df_ri[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].astype(int)
df_ri = df_ri[["date", "ticker", "volume", "adj_final", "r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].rename(columns={"adj_final": "close"})
df_ri

In [13]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)
db.insert_data(table="ta_ripower", df=df_ri)

In [6]:
db.drop_all(table="ta_ripower")